# 📊 Funding Rounds Data Collector
This notebook fetches data from CryptoRank APIs and saves them as CSV files.

In [1]:
import requests
import pandas as pd
from pathlib import Path

# Ensure data folder exists
Path('../data').mkdir(parents=True, exist_ok=True)

## 🔹 Total Funding Overview

In [2]:
url = "https://api.cryptorank.io/v0/funding-rounds-widgets/total-funding-overview?locale=en"
response = requests.get(url)
data = response.json()

flat_data = {
    "fundingRoundsTotal": data.get("fundingRoundsTotal"),
    "fundingRoundsTotalChange": data.get("fundingRoundsTotalChange"),
    "totalInvestments": data.get("totalInvestments"),
    "totalInvestmentsChange": data.get("totalInvestmentsChange"),
    "investmentFocus_id": data.get("investmentsFocus", {}).get("id"),
    "investmentFocus_name": data.get("investmentsFocus", {}).get("name"),
    "investmentFocus_description": data.get("investmentsFocus", {}).get("description"),
    "avgRoundSize_raise_from": data.get("avgRoundSize", {}).get("raise_from"),
    "avgRoundSize_raise_to": data.get("avgRoundSize", {}).get("raise_to"),
    "investmentStage": data.get("investmentStage")
}
df_overview = pd.DataFrame([flat_data])
df_overview.to_csv("../data/total_funding_overview.csv", index=False)
df_overview

,fundingRoundsTotal,fundingRoundsTotalChange,totalInvestments,totalInvestmentsChange,investmentFocus_id,investmentFocus_name,investmentFocus_description,avgRoundSize_raise_from,avgRoundSize_raise_to,investmentStage
0,141,-12.96,4924535500,276.54,17,Payments,Projects that enable payments or create infras...,3000000,10000000,SEED


## 🔹 Hot Rounds

In [3]:
url = "https://api.cryptorank.io/v0/funding-rounds-widgets/hot-rounds/"
response = requests.get(url)
data = response.json()

records = []
for item in data:
    for fund in item.get("funds", []):
        records.append({
            "coinKey": item["coinKey"],
            "coinName": item["coinName"],
            "raise": item["raise"],
            "coinImage": item["coinImage"],
            "fund_slug": fund["slug"],
            "fund_logo": fund["logo"],
            "isHidden": item.get("isHidden", False)
        })

df_hot_rounds = pd.DataFrame(records)
df_hot_rounds.to_csv("../data/hot_rounds.csv", index=False)
df_hot_rounds.head()

,coinKey,coinName,raise,coinImage,fund_slug,fund_logo,isHidden
0,walrus-protocol,Walrus,140000000,https://img.cryptorank.io/coins/60x60.walrus17...,standard-crypto,https://img.cryptorank.io/funds/60x60.standard...,False
1,walrus-protocol,Walrus,140000000,https://img.cryptorank.io/coins/60x60.walrus17...,andreessen-horowitz,https://img.cryptorank.io/funds/60x60.andreess...,False
2,walrus-protocol,Walrus,140000000,https://img.cryptorank.io/coins/60x60.walrus17...,electric-capital,https://img.cryptorank.io/funds/60x60.electric...,False
3,chronicle-labs,Chronicle,12000000,https://img.cryptorank.io/coins/60x60.chronicl...,strobe-ventures,https://img.cryptorank.io/funds/60x60.strobe_p...,False
4,chronicle-labs,Chronicle,12000000,https://img.cryptorank.io/coins/60x60.chronicl...,galaxy-digital,https://img.cryptorank.io/funds/60x60.galaxy16...,False


## 🔹 Fundraising Digest

In [4]:
url = "https://api.cryptorank.io/v0/funding-rounds-widgets/fundraising-digest?locale=en"
response = requests.get(url)
data = response.json()

records = []
for item in data:
    records.append({
        "title": item["title"],
        "publishDate": item["publishDate"],
        "url": item["url"],
        "type": item["type"],
        "subtype": item.get("subtype"),
        "tags": ", ".join([tag["name"] for tag in item.get("tags", [])])
    })

df_digest = pd.DataFrame(records)
df_digest.to_csv("../data/fundraising_digest.csv", index=False)
df_digest.head()

,title,publishDate,url,type,subtype,tags
0,Janover stock soars 300% as former Kraken exec...,2025-04-07T17:44:16.000Z,b96dd-janover-stock-soars-300-as-former-kraken...,news,None,Investments
1,SignalFire Closes On $1B To Invest In Applied AI,2025-04-07T14:00:46.000Z,030c2-signalfire-new-fund-seed-early-stage-app...,news,None,seed funding
2,XRP shines amid $240 million crypto outflows t...,2025-04-07T11:18:09.000Z,f1e1e-xrp-shines-amid-240-million-crypto-outfl...,news,None,Investments
3,North America Startup Investment Spiked In Q1 ...,2025-04-07T11:00:20.000Z,e0349-north-american-startup-investment-spiked...,news,None,seed funding
4,Texas Startups Across Sectors Attract Higher F...,2025-04-07T11:00:08.000Z,984c1-texas-startup-funding-q1-2025-saronic,news,None,venture funding


In [5]:

import requests
import pandas as pd

# Define API endpoint
url = "https://api.cryptorank.io/v0/funding-rounds-v2/"

# Prepare the payload and headers
payload = {
    "limit": 1000,  # Limit the results
    "offset": 0,
}
headers = {
    "Content-Type": "application/json",
    "Accept": "application/json"
}

# Make the POST request
response = requests.post(url, json=payload, headers=headers)
data = response.json()

# Extract and normalize the data
rounds = pd.json_normalize(data['data'])

# Save to CSV for visualization in Django
rounds.to_csv("../data/cryptorank_funding_rounds.csv", index=False)
rounds.head()


,icon,key,name,symbol,date,raise,publicSalesRaise,totalRaise,stage,country,...,valuation,funds,hasFundingRounds,createdAt,isHidden,category.name,category.key,country.key,country.name,category
0,https://img.cryptorank.io/coins/60x60.echooo16...,echooo,Echooo,None,2023-07-17T00:00:00.000Z,15000000.0,NaN,15000000.0,Undisclosed,NaN,...,100000000.0,"[{'name': 'A&T Capital', 'key': 'at-capital', ...",False,2023-07-19T09:08:02.728Z,False,Blockchain Service,blockchain-service,NaN,NaN,NaN
1,https://img.cryptorank.io/coins/60x60.clockwor...,clockwork,Clockwork,None,2022-08-30T00:00:00.000Z,4000000.0,NaN,4000000.0,SEED,NaN,...,NaN,"[{'name': 'Multicoin Capital', 'key': 'multico...",False,2022-08-31T06:26:01.149Z,False,Blockchain Infrastructure,blockchain-infrastructure,US,United States,NaN
2,https://img.cryptorank.io/coins/60x60.tokenpro...,tokenproof,Tokenproof,None,2022-08-30T00:00:00.000Z,5000000.0,NaN,5000000.0,SEED,NaN,...,NaN,"[{'name': 'Penny Jar Capital', 'key': 'penny-j...",False,2022-08-31T08:03:41.461Z,False,NFT,non-fungible-tokens-nft,US,United States,NaN
3,https://img.cryptorank.io/coins/60x60.ether_ma...,ethermail,EtherMail,EMT,2022-08-11T00:00:00.000Z,3000000.0,60000.0,7060000.0,SEED,NaN,...,NaN,"[{'name': 'Fabric Ventures', 'key': 'fabric-ve...",True,2022-08-31T10:50:24.972Z,False,Blockchain Service,blockchain-service,LI,Liechtenstein,NaN
4,https://img.cryptorank.io/coins/60x60.unlockd1...,unlockd,Unlockd,UNLK,2022-08-18T00:00:00.000Z,4440000.0,NaN,4440000.0,SEED,NaN,...,NaN,"[{'name': 'Blockchain Capital', 'key': 'blockc...",False,2022-08-31T13:17:37.878Z,False,DeFi,defi,ES,Spain,NaN


In [6]:
import requests
import pandas as pd

# Prepare the payload and headers
url = "https://api.cryptorank.io/v0/funding-rounds-v2/"
payload = {
    "limit": 1000,
    "offset": 0
}
headers = {
    "accept": "application/json",
    "Content-Type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)
response.raise_for_status()

raw = response.json()["data"]

# ===================== Main Flattened Records =====================
main_records = []
for item in raw:
    main_records.append({
        "coinKey": item.get("key"),
        "coinName": item.get("name"),
        "coinImage": item.get("icon"),
        "raise": item.get("raise"),
        "stage": item.get("stage"),
        "date": item.get("date"),
        "valuation": item.get("valuation"),
        "followersCount": item.get("followersCount"),
        "twitterScore": item.get("twitterScore"),
        "status": item.get("status"),
        "isHidden": item.get("isHidden"),
        "country_name": item["country"]["name"] if item.get("country") else None,
        "country_key": item["country"]["key"] if item.get("country") else None,
        "category_name": item["category"]["name"] if item.get("category") else None,
        "category_key": item["category"]["key"] if item.get("category") else None
    })

# ===================== Flatten Funds =====================
funds_records = []
for item in raw:
    for fund in item.get("funds", []):
        funds_records.append({
            "coinKey": item["key"],
            "coinName": item["name"],
            "raise": item["raise"],
            "coinImage": item["icon"],
            "fund_name": fund.get("name"),
            "fund_key": fund.get("key"),
            "fund_image": fund.get("image"),
            "fund_tier": fund.get("tier"),
            "fund_type": fund.get("type"),
            "fund_category": fund["category"]["name"] if fund.get("category") else None,
            "fund_category_slug": fund["category"]["slug"] if fund.get("category") else None,
            "fund_totalInvestments": fund.get("totalInvestments"),
        })

# ===================== Flatten Top Followers =====================
follower_records = []
for item in raw:
    for follower in item.get("topFollowers", []):
        follower_records.append({
            "coinKey": item["key"],
            "coinName": item["name"],
            "follower_id": follower.get("id"),
            "follower_name": follower.get("name"),
            "follower_username": follower.get("username"),
            "follower_image": follower.get("image"),
            "follower_twitterScore": follower.get("twitterScore"),
            "follower_followersCount": follower.get("followersCount"),
            "relatedEntity_fund": follower.get("relatedEntity", {}).get("fund") if follower.get("relatedEntity") else None,
            "relatedEntity_coin": follower.get("relatedEntity", {}).get("coin") if follower.get("relatedEntity") else None
        })

# Save to CSV
pd.DataFrame(main_records).to_csv("../data/funding_main.csv", index=False)
pd.DataFrame(funds_records).to_csv("../data/funding_funds.csv", index=False)
pd.DataFrame(follower_records).to_csv("../data/funding_followers.csv", index=False)
